In [41]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

**1. напишіть функцію гіпотези лінійної регресії у векторному вигляді**

In [2]:
def linear_regression_hypothesis(X, w):
    return np.dot(X, w)

In [3]:
np.random.seed(0)
X = np.random.rand(50, 2) 
w = np.random.rand(2)
data_predictions = linear_regression_hypothesis(X, w)
data_predictions

array([0.5651017 , 0.55568579, 0.46155679, 0.53738967, 0.75671882,
       0.67945017, 0.63494847, 0.07167502, 0.23851836, 0.76235773,
       0.87910288, 0.52354744, 0.25295214, 0.35223537, 0.46567945,
       0.38836944, 0.46266794, 0.17950255, 0.5814657 , 0.82378497,
       0.36168251, 0.48912732, 0.63302308, 0.17741191, 0.31200736,
       0.50491471, 0.69748904, 0.18513498, 0.51107838, 0.38207001,
       0.13755438, 0.48218155, 0.23280551, 0.58270091, 0.59392025,
       0.78839993, 0.82537793, 0.5116661 , 0.22414527, 0.23278615,
       0.32738857, 0.23045323, 0.45570911, 0.38003087, 0.64130344,
       0.39613743, 0.28274883, 0.24562736, 0.40297736, 0.56313708])

**2. створіть функцію для обчислення функції втрат у векторному вигляді**

In [4]:
def loss_function(X, y, w):
    m = X.shape[0]
    return np.square(linear_regression_hypothesis(X, w) - y).sum() / (2*m)

In [5]:
np.random.seed(0)
X = np.random.rand(50, 2) 
w = np.random.rand(2)
y = np.random.rand(50)

test_loss_function = loss_function(X, y, w)
test_loss_function


0.060349101872647956

**3. реалізуйте один крок градієнтного спуску**

In [19]:
def step(w, grad_w, learning_rate=0.001):
    w = w - learning_rate * grad_w
    return w, grad_w

def one_step_of_gradient_descent(X, y, w, num_iter=10000, learning_rate=0.001, epsilon=0.0000001):
    loss_history = []
    
    for i in range(num_iter):
        m = len(X)
        hypothesis = np.dot(X, w)
        error = hypothesis - y
        gradient = np.dot(X.T, error) / m
        w, grad_w = step(w, gradient, learning_rate=learning_rate)
        
        loss = np.mean(np.square(error))
        loss_history.append(loss)
        
        if len(loss_history) > 1 and abs(loss - loss_history[-2]) < epsilon:
            break
        
    return w, grad_w, loss_history

**3. знайдіть найкращі параметри дw  для датасету прогнозуючу ціну на будинок залежно від площі
 кількості ванних кімнат та кількостіспалень**ь

In [7]:
data_house_cost = pd.read_csv("Housing.csv")
data_house_cost.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [8]:
data_house_cost.tail()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished
544,1750000,3850,3,1,2,yes,no,no,no,no,0,no,unfurnished


In [25]:
scaler = StandardScaler()

columns = ['area', 'bedrooms', 'bathrooms', 'price']
result_data = scaler.fit_transform(data_house_cost[columns])
normalized_data = pd.DataFrame(result_data, columns=columns)

normalized_data

,area,bedrooms,bathrooms,price
0,1.046726,1.403419,1.421812,4.566365
1,1.757010,1.403419,5.405809,4.004484
2,2.218232,0.047278,1.421812,4.004484
3,1.083624,1.403419,1.421812,3.985755
4,1.046726,1.403419,-0.570187,3.554979
...,...,...,...,...
540,-0.991879,-1.308863,-0.570187,-1.576868
541,-1.268613,0.047278,-0.570187,-1.605149
542,-0.705921,-1.308863,-0.570187,-1.614327
543,-1.033389,0.047278,-0.570187,-1.614327


In [33]:
X = normalized_data[['area', 'bathrooms', 'bedrooms']].values
X = np.hstack((np.ones((X.shape[0], 1)) , X))
y = normalized_data["price"].values.reshape(-1, 1)

n = X.shape[1]
w = np.linspace(0, 0, n).reshape((n, 1))
w, best_features_w, data_lost = one_step_of_gradient_descent(X, y, w, 10000, learning_rate=0.001)
loss = data_lost[-1]

print('Parameters searched for w: ', best_features_w.flatten())
print('Optimal loss parameter: ', loss)

Parameters searched for w:  [-3.17788609e-17 -4.33417916e-03 -3.33958730e-03  4.46421481e-03]
Optimal loss parameter:  0.5129846234686081


**знайдіть ці ж параметри за допомогою аналітичного рішення**

In [35]:
analytical_solution_w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
analytical_solution_loss = loss_function(X, y, analytical_solution_w)

print('Parameters searched for w: ', analytical_solution_w.flatten())
print('Optimal loss parameter: ', analytical_solution_loss)

Parameters searched for w:  [3.15434935e-16 4.39452085e-01 3.72344423e-01 1.60528660e-01]
Optimal loss parameter:  0.2564584666470618


**порівняйте отримані результати**

In [40]:
print('Gradient descent:')
print('Parameters searched for w: ', best_features_w.flatten())
print('Optimal loss parameter: ', loss)

print('')
print('Analytical solution:')
print('Parameters searched for w: ', analytical_solution_w.flatten())
print('Optimal loss parameter: ', analytical_solution_loss)

difference_w = best_features_w.flatten() - analytical_solution_w.flatten()
difference_loss = np.abs(loss - analytical_solution_loss)

print('')
print('Difference in parameters w: ', difference_w)
print('Difference in losses: ', difference_loss)

Gradient descent:
Parameters searched for w:  [-3.17788609e-17 -4.33417916e-03 -3.33958730e-03  4.46421481e-03]
Optimal loss parameter:  0.5129846234686081

Analytical solution:
Parameters searched for w:  [3.15434935e-16 4.39452085e-01 3.72344423e-01 1.60528660e-01]
Optimal loss parameter:  0.2564584666470618

Difference in parameters w:  [-3.47213796e-16 -4.43786264e-01 -3.75684010e-01 -1.56064445e-01]
Difference in losses:  0.2565261568215463
